<a href="https://colab.research.google.com/github/therealthaibinh/reference-creator/blob/master/FactCheck_references.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run this cell

In [5]:
from IPython.core.display import display, HTML
import ipywidgets as widgets
# from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from ipywidgets import interact, Layout


from datetime import datetime

try:
    import requests
except:
    !pip install requests
    import requests
    
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup

def create_factcheck_ref(url):
    page = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Author
    lstAuthors = []
    for strAuthor in page.findAll(attrs={'class' : 'author url fn'}):
        lstAuthors.append(strAuthor.contents[0])
    if len(lstAuthors)==1:
        strFirstAuthor = lstAuthors[0]
        strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')]
    elif len(lstAuthors)==2:
        strFirstAuthor = lstAuthors[0]
        strSecondAuthor = lstAuthors[1]
        strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')] + ' and ' +\
                    strSecondAuthor[strSecondAuthor.find(' ')+1:] + ', ' + strSecondAuthor[:strSecondAuthor.find(' ')]
    else:
        strFirstAuthor = lstAuthors[0]
        strAuthor = strFirstAuthor[strFirstAuthor.find(' ')+1:] + ', ' + strFirstAuthor[:strFirstAuthor.find(' ')] + ' et. al'

    # Title
    strTitle = page.find(attrs={'class' : 'entry-title'}).contents[0]
    link_tagged = "<a href='{href}'>"+strTitle+"</a>"

    # Date
    strDate = page.find(attrs={'class' : 'entry-date published updated'}).contents[0]
    strDate = datetime.strptime(strDate, '%B %d, %Y').strftime("%-d %b %Y")
    
    return {'AUTHOR':strAuthor, 'TITLE':strTitle, 'DATE':strDate}

def f(strURLs):
    lstURL = strURLs.split('\n')
    for url in lstURL:
        try:
            dict_extract = create_factcheck_ref(url)
            link_tagged = "<a href='{href}'>"+dict_extract['TITLE']+"</a>"
            full_tagged = dict_extract['AUTHOR'] +'. "' +\
                            link_tagged + '." FactCheck.org. ' +\
                            dict_extract['DATE'] + '.'
            html = HTML(full_tagged.format(href=url))
            display(html)

        except:
            print("Misformatted URL or maybe some other funky error")


widget_URL = widgets.Textarea(
    value='',
    placeholder='Paste URL, one per line (no need for comma in between)',
    description='URLs',
    disabled=False,
    layout=Layout(width='80%', height='100px')
)

interactive_plot = interact(f, strURLs=widget_URL)



interactive(children=(Textarea(value='', description='URLs', layout=Layout(height='100px', width='80%'), place…